## Create pairs of sketches and pixel art images from 64x64 images taken from TinyHero dataset

- Use gaussian-blur and canny edge detection to generate a viable sketch for the given image
- https://stackabuse.com/opencv-edge-detection-in-python-with-cv2canny/
- https://codewithcurious.com/python-projects/convert-image-into-sketch-python/
- The code below `create_sketch` is derived using experimentation of above 2 code references

In [15]:
import cv2 as cv
import os

def remove_extension(i: str) -> str:
    a, _ = os.path.splitext(i)
    return a

def get_filename(i: str) -> str:
    return os.path.basename(i)

def create_sketch(i: str, levels: int = 3, directory="."):
    image = cv.imread(i)

    # Create a basic sketch
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    invert_image = cv.bitwise_not(gray_image)
    blur_image = cv.GaussianBlur(invert_image, (5,5), 0)
    invert_blur = cv.bitwise_not(blur_image)
    sketch = cv.divide(gray_image, invert_blur, scale=256.0)    
    
    # perform edge detection on the sketch
    b1 = cv.GaussianBlur(sketch, (5, 5), 0)
    b2 = b1
    for _ in range(0, levels - 1):
        b2 = cv.GaussianBlur(b2, (5, 5), 0)
    edge = cv.Canny(b2, 50, 100)
    edge_inv = cv.bitwise_not(edge)
    
    file_path = os.path.join(directory, f"{get_filename(remove_extension(i))}_s{levels}.png")
    cv.imwrite(file_path, edge_inv)

## Generate data for training a Pix2Pix model

In [16]:
import glob
import shutil

os.makedirs("./pairs", exist_ok=True)

total_levels = 3

for i, f in enumerate(glob.glob("./input/*.png")):
    target = f"{i:03}"
    target = os.path.join("./pairs", target) + ".png"
    shutil.copyfile(f, target)
    for j in range(1, total_levels + 1):
        create_sketch(target, levels = j, directory = "./pairs")

### Create folder structure for pix2pix

In [17]:
total_files = i

test = int(total_files * 0.79)
val = int(total_files * 0.90)

splits = ["train", "test", "val"]

for s in splits:
    os.makedirs(f"./pix2pix_data/A/{s}", exist_ok=True)
    os.makedirs(f"./pix2pix_data/B/{s}", exist_ok=True)

data_number = 0

for i in range(0, total_files):
    current_split = "train"
    if test < i < val:
        current_split = "test"
    elif val <= i:
        current_split = "val"
    pair_name = f"{i:03}"
    pair_A_path = os.path.join("./pairs", pair_name) + ".png"
    for j in range(1, total_levels + 1):
        # sketch files 1, 2, 3
        pair_B_path = os.path.join("./pairs", f"{pair_name}_s{j}.png")
        # file naming
        data_name = f"{data_number:03}"
        data_A_path = os.path.join("./pix2pix_data/A/", current_split, data_name) + ".png"
        data_B_path = os.path.join("./pix2pix_data/B/", current_split, data_name) + ".png"
        shutil.copyfile(pair_A_path, data_A_path)
        shutil.copyfile(pair_B_path, data_B_path)
        data_number += 1

# Cleanup pairs folder as it is no longer needed
shutil.rmtree("pairs")

## Convert to combined AB images

In [20]:
os.makedirs("./pix2pix_data/AB")
!python ./pix2pix/datasets/combine_A_and_B.py --fold_A ./pix2pix_data/A --fold_B ./pix2pix_data/B --fold_AB ./pix2pix_data/AB

[fold_A] =  ./pix2pix_data/A
[fold_B] =  ./pix2pix_data/B
[fold_AB] =  ./pix2pix_data/AB
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = test, use 108/108 images
split = test, number of images = 108
split = train, use 795/795 images
split = train, number of images = 795
split = val, use 102/102 images
split = val, number of images = 102
